## Table of Contents

<div class="alert alert-success">



<h4><a href="#item0">PART A : </a> </h4>    
<h4><a href="#item1">1) Download and Clean Dataset</a> </h4>
<h4><a href="#item2">2) Import Keras and necessary libraries</a></h4>
<h4><a href="#item3">3) Build a Neural Network as a function</a></h4>
<h4><a href="#item4">4) Split the data set with (train_test_split) method</a> </h4>
<h4><a href="#item5">5) Create and Train the model with splitted Dataset</a> </h4>
<h4><a href="#item6">6) Use the trained model to make predictions</a></h4>
<h4><a href="#item7">7) Calling Evaluation metric (mean_squared_error)</a></h4>
<h4><a href="#item8">8) 50 times iteration (list) of mean_squared_error(s)</a> </h4>
   
<h4><a href="#item9">PART B. Normalize the data and recreate the mean_squared_error list</a></h4>
<h4><a href="#item10">PART C. Increase the number of epochs and recreate the mean_squared_error list: </a></h4>
<h4><a href="#item11">PART D. Increase the number of hidden layers and recreate the mean_squared_error list</a></h4>   
<h4><a href="#item12">Conclusion about all comparisons </a> <br>      
   
    
 

</div>

<a id="item1"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [9]:
n_cols = predictors.shape[1] # number of predictors

In [10]:
n_cols

8

<a id='item2'></a>

## Import Keras and necessary libraries

In [11]:
import keras

Using TensorFlow backend.


In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

<a id='item3'></a>

## Build a Neural Network as a function

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layer, with 10 nodes in it.

<a id='item4'></a>

## Split the data set 

we will create the training and testing data sets from the NOT normalized Data set with help of train_test_split method from the sklearn library.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
print (predictors.shape, target.shape)

(1030, 8) (1030,)


In [16]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(721, 8) (721,)
(309, 8) (309,)


<a id='item5'></a>

## Create and Train the model with splitted Dataset

Let's call the function now to create our model.

In [17]:
# call function to build the model
model = regression_model()

In [18]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 0s - loss: 123756.9259
Epoch 2/50
 - 0s - loss: 72420.7791
Epoch 3/50
 - 0s - loss: 36865.1081
Epoch 4/50
 - 0s - loss: 15367.0531
Epoch 5/50
 - 0s - loss: 7135.9295
Epoch 6/50
 - 0s - loss: 5572.2632
Epoch 7/50
 - 0s - loss: 5348.6983
Epoch 8/50
 - 0s - loss: 5127.8170
Epoch 9/50
 - 0s - loss: 4939.0108
Epoch 10/50
 - 0s - loss: 4748.4994
Epoch 11/50
 - 0s - loss: 4560.9279
Epoch 12/50
 - 0s - loss: 4383.5489
Epoch 13/50
 - 0s - loss: 4201.5667
Epoch 14/50
 - 0s - loss: 4019.8081
Epoch 15/50
 - 0s - loss: 3843.0980
Epoch 16/50
 - 0s - loss: 3677.4389
Epoch 17/50
 - 0s - loss: 3508.1363
Epoch 18/50
 - 0s - loss: 3348.6563
Epoch 19/50
 - 0s - loss: 3192.0820
Epoch 20/50
 - 0s - loss: 3039.6406
Epoch 21/50
 - 0s - loss: 2888.3033
Epoch 22/50
 - 0s - loss: 2743.9741
Epoch 23/50
 - 0s - loss: 2603.7904
Epoch 24/50
 - 0s - loss: 2457.8815
Epoch 25/50
 - 0s - loss: 2322.1306
Epoch 26/50
 - 0s - loss: 2151.4855
Epoch 27/50
 - 0s - loss: 1766.8739
Epoch 28/50
 - 0s - loss: 1331.2

<a id='item6'></a>

## Use the trained model to make predictions

In [19]:
predictions = model.predict(X_test)

<a id='item7'></a>

## Calling Evaluation metric (mean_squared_error)

In this stage we will discover how good our model predicted the target 'Strength' values using the 30 % of non trained data..

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
# mean_squared_error(y_true, y_pred)
print ('Mean Squared Error after one time randomly splitting the dataset :')
mean_squared_error(y_test, predictions)

Mean Squared Error after one time randomly splitting the dataset :


158.14457897521854

<a id='item8'></a>

## 50 times iteration (list) of mean_squared_error(s)

In this part we will create a function to make all the routin steps above 50 times for us. And We will append these iteration results of mean_squared_errors in to a list.

In [22]:
#lets use the statistics library for the mean and standart deviation of a list
import statistics

In [23]:
def iter_func_A(n) :
    errors_as_list=[]
    for i in range(n):
       X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
       model = regression_model()
       model.fit(X_train, y_train, epochs=50, verbose=0)
       predictions = model.predict(X_test)
       errors_as_list.append(mean_squared_error(y_test, predictions))
       
    print("Mean Square Errors from the all iterations : ", errors_as_list,'\n\n')
  
    m1 = statistics.mean(errors_as_list)
    s1 = statistics.stdev(errors_as_list)
  
    # Printing the mean and the standart deviation  
    print("Mean of Mean Square Errors List is :", m1) 
    print("Standart Deviation for the List is :", s1) 
     
        

In [24]:
iter_func_A(50)

Mean Square Errors from the all iterations :  [575.9322139412797, 106.95328418943933, 109.03699429513173, 141.39991880290967, 168.6223920845069, 115.62278560083183, 911.5817659236894, 323.47318536762594, 201.9716693824651, 102.3871412653894, 114.62378246893256, 638.6050701569908, 1479.2677192191904, 526.4687481484084, 635.1366464155585, 304.31142903908534, 546.2934789012413, 110.35740654238629, 111.04320067064415, 467.20060226427915, 125.02765623606648, 366.42341257187326, 626.1526095353521, 309.43787663965554, 107.47155495802471, 135.79740410365966, 104.55693390257376, 127.55003455286734, 96.27089095943614, 1503.7372014719674, 244.6687258292423, 200.19556946418436, 873.8199177214589, 470.67415330746223, 110.8854943672288, 357.67486023361147, 122.21210303247385, 375.4422353066023, 135.47660435531594, 119.19018537896432, 555.4311491030534, 248.5605976468823, 992.5353529422135, 364.0688263858331, 130.79055965898766, 119.79300730202564, 239.09126611407564, 119.00985772650306, 166.62952657

<hr>



<a id='item9'></a>

## PART B. Normalize the data and recreate the mean_squared_error list

We will normalize the data by substracting the mean and dividing by the standard deviation.

In [25]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Lets modify our iteration function slightly :

In [26]:
def iter_func_B(n) :
    errors_as_list=[]
    for i in range(n):
       X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
       model = regression_model()
       model.fit(X_train, y_train, epochs=50, verbose=0)
       predictions = model.predict(X_test)
       errors_as_list.append(mean_squared_error(y_test, predictions))
       
    print("Mean Square Errors from the all iterations : ", errors_as_list,'\n\n')
  
    m1 = statistics.mean(errors_as_list)
    s1 = statistics.stdev(errors_as_list)
  
    # Printing the mean and the standart deviation  
    print("Mean of Mean Square Errors List is :", m1) 
    print("Standart Deviation for the List is :", s1) 

In [27]:
iter_func_B(50)

Mean Square Errors from the all iterations :  [318.7238796752508, 352.3309017894599, 392.2033875657034, 380.8106224562341, 292.6831706015439, 293.66222238417816, 248.1720483235584, 410.90329316837665, 240.81098317793405, 298.91439278642383, 333.0910030494724, 274.040683629756, 377.3909583644935, 264.32876634133885, 273.8737932159769, 553.3483733931295, 398.4701114141877, 342.59128618335757, 348.9137132850817, 292.2843880493782, 287.2352036222574, 378.06720482371793, 229.9588260957549, 316.4766503431158, 245.83234854018158, 252.54190254784027, 472.4877077181783, 267.81013104689646, 294.717246710222, 274.80418461763327, 536.7657563079723, 428.75405391900705, 348.26842977374383, 232.57694320540543, 285.7944411247693, 362.18875205931266, 291.3960103457643, 292.95627869228275, 355.6443840739609, 327.9083944914016, 353.2356500774474, 344.4129892900272, 402.13735848091403, 473.0695129217654, 279.1017427219402, 358.9579945298266, 398.48272670930686, 670.2636250252078, 279.0397545747755, 406.25

#### Compare the mean of the mean squared errors Between Step A and Step B :

We see here the mean of the mean squared errors are reduced slightly from 361.5 to 342.7 .

<hr>



<a id='item10'></a>

## PART C. Increase the number of epochs and recreate the mean_squared_error list:


Again after modifying our iteration function for epochs from 50 to 100 :

In [28]:
def iter_func_C(n) :
    errors_as_list=[]
    for i in range(n):
       X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
       model = regression_model()
       model.fit(X_train, y_train, epochs=100, verbose=0)
       predictions = model.predict(X_test)
       errors_as_list.append(mean_squared_error(y_test, predictions))
       
    print("Mean Square Errors from the all iterations : ", errors_as_list,'\n\n')
  
    m1 = statistics.mean(errors_as_list)
    s1 = statistics.stdev(errors_as_list)
  
    # Printing the mean and the standart deviation  
    print("Mean of Mean Square Errors List is :", m1) 
    print("Standart Deviation for the List is :", s1) 

In [29]:
iter_func_C(50)

Mean Square Errors from the all iterations :  [139.22057937963436, 148.4035036714036, 171.25417219888098, 145.18405744389597, 154.51950345901537, 164.69570817668642, 178.71590691155527, 159.85342167680545, 160.7663909842186, 162.87021187506164, 168.8393914764568, 152.98268002675965, 157.11363342071505, 162.3904595288734, 156.54517922614497, 235.67861726688798, 197.09758280212796, 153.55575082238377, 174.45933562650978, 200.64639874002015, 169.92773957769785, 157.7653243583448, 186.36623296729718, 165.75799744548453, 184.8827704960779, 196.41000327311625, 152.4584424762532, 169.36474731697749, 173.8952940650637, 147.52675664441642, 189.33911441372715, 158.35252962793362, 165.63207498070355, 242.3450436477737, 151.34053058018685, 152.201235896393, 155.24630671151402, 155.14619295345634, 149.16065922871613, 150.7591966551579, 136.47121790132573, 164.41879621849864, 158.7390761248328, 169.7885086934789, 174.4075092390793, 200.63129791258635, 158.05142725006323, 158.2798893769147, 160.24607

#### Compare the mean of the mean squared errors Between Step C and Step B :

We see here the mean of the mean squared errors are reduced better. From 342.7 to 167.5 . It is nearly 51 % decrease on  error rate.

<hr>



<a id='item11'></a>

## PART D. Increase the number of hidden layers and recreate the mean_squared_error list

This time we will cmodify the model function in order to build 3 hidden layers, each of 10 nodes and ReLU activation function.

In [30]:
# define regression model again
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [32]:
iter_func_B(50)

Mean Square Errors from the all iterations :  [98.26700079307065, 140.12268916325635, 138.3819512841003, 138.7671861536702, 133.9199881766596, 128.88278653037602, 118.52890846843201, 141.1574372147391, 141.8491532375119, 160.88251131312228, 104.32016515697954, 139.9097625303441, 134.76813301069447, 123.10278297372962, 139.3833704655947, 115.53598777967936, 127.12276942942835, 113.24072858823276, 122.31249835598314, 81.41190028248425, 134.6568940898554, 114.51220904180884, 138.94474404254203, 115.97813006852628, 128.8044753606979, 124.98055360035154, 150.54156592543043, 128.30595470107014, 131.79274491518868, 143.56239156377927, 106.43850711279205, 128.69380071765502, 144.91211831571783, 132.44038177334528, 145.37666000288016, 124.584194114178, 125.94380207134672, 149.26761345956115, 118.14569142563177, 115.71196056049065, 132.15044545071453, 137.55999874332866, 139.37089056895817, 119.17587965314675, 135.11088800570354, 90.47852828833179, 140.82771286584412, 157.02653242993807, 122.216

#### Compare the mean of the mean squared errors Between Step D and Step B :

We see here the mean of the mean squared errors are reduced most. from 342.7 to 129 . It is a good 62.3 % decrease on  error rate.

<hr>



<a id='item12'></a>

### Conclusion about all comparisons :

Part A --------> Mean of Mean Square Errors List is : 361.53 <br>
Part B --------> Mean of Mean Square Errors List is : 342.69 <br>
Part C --------> Mean of Mean Square Errors List is : 167.47 <br>
Part D --------> Mean of Mean Square Errors List is : 129.06 <br>
    

From the above results we can conclude that :  <br>Normalizing the data set didnt bring too much in this special dataset (From A to B) <br>
Increasing epochs from 50 to 100 was the most efficient way to decrease mean(mean_squared_errors)(From B to C)...<br>
And finally trippling the hidden layers (From B to D), eventhough improved the prediction accuracy but in the cost of computing time.It took too long to train 3 hidden layers with 10 nodes each .


<hr>



### This notebook was created by Uygar Hizal.

### Thank you for reading !